## Example of training/testing using deterministic models with pytorch-lightning

In [7]:
from pathlib import Path
from types import SimpleNamespace
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: # To be able to import files 
    sys.path.append(module_path) # without installing the project

from bayesrul.cmapss.dataset import CMAPSSDataModule
from bayesrul.cmapss.models import CMAPSSModel, get_checkpoint, TBLogger

In [8]:
args = SimpleNamespace(
    data_path="../data/cmapss/",
    out_path="../results/cmapss/",
    scn="dnn_ptl",
    net="linear",
)
checkpoint_dir = Path(f"{args.out_path}/{args.scn}/checkpoints/{args.net}")
logger = TBLogger(
    f"{args.out_path}/{args.scn}/lightning_logs/{args.net}",
    default_hp_metric=False,
)

In [10]:
# Just to illustrate. To properly train see scripts/cmapss_training.py
data = CMAPSSDataModule(args.data_path, batch_size=10000)
dnn = CMAPSSModel(data.win_length, data.n_features, args.net)

checkpoint_file = get_checkpoint(checkpoint_dir)
monitor = "loss/val"
checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_dir, monitor=monitor)
earlystopping_callback = EarlyStopping(monitor=monitor, patience=20)

trainer = pl.Trainer(
    gpus=[0],
    max_epochs=10,
    log_every_n_steps=2,
    logger=logger,
    callbacks=[
        checkpoint_callback,
        earlystopping_callback,
    ],
)
trainer.fit(dnn, data, ckpt_path=checkpoint_file)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/arthur/.conda/envs/bayesrul/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /home/arthur/dev/bayesrul/results/cmapss/dnn_ptl/checkpoints/linear exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at ../results/cmapss/dnn_ptl/checkpoints/linear/epoch=9-step=20.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name | Type   | Params
--------------------------------
0 | net  | Linear | 45.6 K
--------------------------------
45.6 K    Trainable params
0         Non-trainable params
45.6 K    Total params
0.182     Total estimated model params size (MB)
Restored all states from the checkpoint file at ../results/cmapss/dnn_ptl/checkpoints/linear/epoch=9-step=20.ckpt


In [11]:
data = CMAPSSDataModule(args.data_path, batch_size=1000)
dnn = CMAPSSModel.load_from_checkpoint(get_checkpoint(checkpoint_dir))
trainer = pl.Trainer(gpus=[0], log_every_n_steps=10, logger=logger)
trainer.test(dnn, data, verbose=False)

/home/arthur/.conda/envs/bayesrul/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Testing: 0it [00:00, ?it/s]

[{'loss/test': 5722.74853515625}]

In [12]:
dl = data.train_dataloader()
print(f"Win length {data.win_length}, n_features {data.n_features}")
ex = next(iter(dl))

Win length 18, n_features 14
